In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
X_train = pd.read_csv("../datasets/complete_data/Train_DB.csv")
X_test = pd.read_csv("../datasets/complete_data/Test_DB.csv")

/home/milad/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(X_test.shape)
print(X_train.shape)

(128, 24)
(42208, 27)


In [4]:
X_train["date"] = pd.to_datetime(X_train["date"] )
X_test["Year_com"] = pd.to_datetime(X_test["Year_com"] )


X_train['Year'] = X_train.date.dt.year
X_train['Month'] = X_train.date.dt.month
X_train['day'] = X_train.date.dt.day



X_test['Year'] = X_test.Year_com.dt.year
X_test['Month'] = X_test.Year_com.dt.month
X_test['day'] = X_test.Year_com.dt.day


X_train= X_train.drop(['date'],axis=1)
X_test = X_test.drop(['Year_com'],axis=1)


In [5]:
y_train =X_train['Winner_num']
y_test =X_test['Winner_num']


X_train= X_train.drop(['Unnamed: 0','home_score','away_score','tournament','winner','city','country','Winner_num'],axis=1)
X_test = X_test.drop(['Unnamed: 0','Score1','Score2','winner','Winner_num'],axis=1)

In [6]:
X_train.head()

,home_team,away_team,RK1,RC1,TP1,Cur_weighted1,last_weighted1,RK2,RC2,TP2,...,last_weighted2,NW1,NF1,ND1,NW2,NF2,ND2,Year,Month,day
0,Guinea,Sierra Leone,65.0,-2.0,0.0,0.0,0.0,86.0,-7.0,0.0,...,0.0,0,0,0,0,0,0,1993,8,9
1,Faroe Islands,Norway,111.0,-17.0,0.0,0.0,0.0,9.0,5.0,0.0,...,0.0,0,0,0,0,0,0,1993,8,11
2,Sweden,Switzerland,4.0,0.0,0.0,0.0,0.0,3.0,9.0,0.0,...,0.0,0,0,0,0,0,0,1993,8,11
3,Australia,Canada,52.0,-7.0,0.0,0.0,0.0,46.0,11.0,0.0,...,0.0,0,0,0,0,0,0,1993,8,15
4,Bolivia,Ecuador,59.0,28.0,0.0,0.0,0.0,35.0,30.0,0.0,...,0.0,0,0,0,0,0,0,1993,8,15


In [7]:
X_test.head()

,Team1,Team2,RK1,RC1,TP1,Cur_weighted1,last_weighted1,RK2,RC2,TP2,...,last_weighted2,NW1,NF1,ND1,NW2,NF2,ND2,Year,Month,day
0,Russia,Saudi Arabia,70.0,-4.0,456.53,166.07,108.09,67.0,0.0,465.28,...,180.10,47,25,28,41,36,23,2018,6,14
1,Egypt,Uruguay,45.0,1.0,649.43,208.27,291.04,14.0,3.0,1018.41,...,243.84,62,24,14,46,28,26,2018,6,15
2,Morocco,IR Iran,41.0,1.0,685.86,369.10,172.65,37.0,-1.0,708.35,...,246.02,43,26,31,63,13,24,2018,6,15
3,Portugal,Spain,4.0,0.0,1273.69,646.92,345.91,10.0,-2.0,1125.50,...,305.74,59,20,21,70,15,15,2018,6,15
4,France,Australia,7.0,0.0,1198.13,520.12,428.38,36.0,4.0,718.33,...,193.77,61,19,20,47,29,24,2018,6,16


In [8]:
X_train= X_train.drop(['home_team','away_team'],axis=1)
X_test = X_test.drop(['Team1','Team2'],axis=1)

In [9]:
print(y_train.shape)
print(y_test.shape)

(42208,)
(128,)


# Train Model 

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test) 

In [ ]:
from sklearn import svm
model_svm = svm.SVC()
model_svm.fit(X_train,y_train)

y_pred_svm = model_svm.predict(X_test)


accuracy_score(y_test, y_pred_svm)

In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

/home/milad/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/milad/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [11]:
def modelfit(alg, dtrain, dtest, predictors,y_train,y_test,useTrainCV=False, cv_folds=5):
    
    print(dtrain.shape)
    print(dtest.shape)
    
    print(y_train.shape)
    print(y_test.shape)
    
    if useTrainCV == True:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=y_train)
        xgtest = xgb.DMatrix(dtest[predictors].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,metrics='auc')
#         alg.set_params(n_estimators=cvresult.shape[0])
        alg.set_params(n_estimators=dtrain.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], y_train ,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_train, dtrain_predictions))
#     print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))
    
#     Predict on testing data:
    dtest['predprob'] = alg.predict_proba(dtest[predictors])[:,1]
    import numpy as np
    best_preds = np.asarray([np.argmax(line) for line in dtest['predprob']])
#     results = test_results.merge(dtest[['ID','predprob']], on='ID')
#     results = dtest
#     print ('AUC Score (Test): %f' % metrics.roc_auc_score(y_test, results['predprob'],average='weighted',sample_weight=None))
    print ('AUC Score (Test): %f' % metrics.accuracy_score(y_test, best_preds))
                
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [12]:
predictors = [x for x in X_train.columns]

xgb1 = XGBClassifier(
        num_class =3,
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'multi:softmax',
        nthread=4,
        scale_pos_weight=1,
        seed=27)

xgb1.fit(X_train,y_train)
y_predes = xgb1.predict(X_test)
accuracy_score(y_test,y_predes)

# modelfit(xgb1, X_train, X_test, predictors,y_train,y_test)

0.5390625

In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':[3,5,8,9],
    'min_child_weight':[1,2,3],
    'learning_rate':[0.1,0.01],
    'reg_alpha':[0, 0.001, 0.01, 0.05],
    'gamma':[i/10.0 for i in range(0,5)]
    
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(num_class =3, n_estimators=140,
                                        min_child_weight=1, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)